<a href="https://colab.research.google.com/github/athens-21/DE242/blob/main/DE242_2_2024_Feature_Engineering_and_Performance_Evaluation(661010252).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Programmer** : Rattanin Kliangmol

**Date** : 11 mar 2025

In [142]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [143]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.dat'
col_names = [
    'age', 'sex', 'chest_pain_type', 'resting_blood_pressure',
    'serum_cholestoral', 'fasting_blood_sugar', 'resting_electro_results',
    'max_heart_rate', 'exercise_induced_angina', 'oldpeak', 'slop',
    'num_major_vessels', 'thal', 'target'
]
heart_df = pd.read_csv(url, header=None, skiprows=0, names=col_names, sep='\s+')


In [144]:
heart_df.head()

,age,sex,chest_pain_type,resting_blood_pressure,serum_cholestoral,fasting_blood_sugar,resting_electro_results,max_heart_rate,exercise_induced_angina,oldpeak,slop,num_major_vessels,thal,target
0,70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0,3.0,2
1,67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0,7.0,1
2,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,2
3,64.0,1.0,4.0,128.0,263.0,0.0,0.0,105.0,1.0,0.2,2.0,1.0,7.0,1
4,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,1


In [145]:
heart_df['target'].value_counts()

,count
target,
1,150
2,120


In [146]:
heart_df.dtypes

,0
age,float64
sex,float64
chest_pain_type,float64
resting_blood_pressure,float64
serum_cholestoral,float64
fasting_blood_sugar,float64
resting_electro_results,float64
max_heart_rate,float64
exercise_induced_angina,float64
oldpeak,float64


In [147]:
heart_df.isnull().any(axis=1)

,0
0,False
1,False
2,False
3,False
4,False
...,...
265,False
266,False
267,False
268,False


In [148]:
X = heart_df.iloc[:, 0:-1]
X.shape

(270, 13)

In [149]:
y = heart_df.iloc[:, -1]
y.shape

(270,)

In [150]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2025)#test_size=0.2 = 80% training. 20% test

X_train.shape

(216, 13)

In [151]:
X_train.shape #216/270 = 0.8

(216, 13)

In [152]:
y_train.shape

(216,)

In [153]:
X_test.shape   # (54, 13)

(54, 13)

In [154]:
y_test.shape   # (54,)

(54,)

In [155]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# from sklearn.feature_selection import f_classif
# from sklearn.feature_selection import mutual_info_classif

In [156]:
# Create a selector for a particular statistical score
# selector = SelectKBest(score_func=chi2, k=10)
# selector = SelectKBest(score_func=f_classif, k=10)
# selector = SelectKBest(score_func=mutual_info_classif, k=10)

selector = SelectKBest(score_func=chi2, k=10)

# Fit the selector to the training data set.
selector_model = selector.fit(X_train, y_train)

In [157]:
print(selector_model.scores_)

[ 17.72065735   6.42022337  11.02865607  16.14814939  67.83827962
   0.18636911   3.67557018 143.67113172  24.09473425  49.88842338
   4.65669721  57.02034887  49.94260247]


In [158]:
selector_model.get_support(indices=True)

array([ 0,  1,  2,  3,  4,  7,  8,  9, 11, 12])

In [159]:
X_train.columns[selector_model.get_support(indices=True)]

Index(['age', 'sex', 'chest_pain_type', 'resting_blood_pressure',
       'serum_cholestoral', 'max_heart_rate', 'exercise_induced_angina',
       'oldpeak', 'num_major_vessels', 'thal'],
      dtype='object')

In [160]:
print(selector_model.scores_[selector_model.get_support(indices=True)])

[ 17.72065735   6.42022337  11.02865607  16.14814939  67.83827962
 143.67113172  24.09473425  49.88842338  57.02034887  49.94260247]


In [161]:
X_train_model = selector_model.transform(X_train)
X_train_model  # Transformed training data

X_train_model.shape  # (216, 10)


(216, 10)

In [162]:
X_test_model = selector_model.transform(X_test)
X_test_model  # Transformed test data

X_test_model.shape  # (54, 10)

(54, 10)

In [163]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
logisticRegr.fit(X_train_model, y_train)
predictions = logisticRegr.predict(X_test_model)


In [164]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

In [165]:
y_test.values

array([1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2,
       1, 2, 1, 1, 1, 1, 1, 1, 2, 2])

In [166]:
print("Performance:")
print(" - accuracy = " + str(accuracy))
print(" - precision = " + str(precision))
print(" - recall = " + str(recall))
print(" - f1 = " + str(f1))


Performance:
 - accuracy = 0.7962962962962963
 - precision = 0.8387096774193549
 - recall = 0.8125
 - f1 = 0.8253968253968254


In [167]:
from sklearn.feature_selection import mutual_info_regression

selector = SelectKBest(score_func=mutual_info_regression, k=10)

# Fit the selector to the training data set.
selector_model = selector.fit(X_train, y_train)
X_train_model = selector_model.transform(X_train)
X_test_model = selector_model.transform(X_test)

In [168]:
X_train.columns[selector_model.get_support(indices=True)]

Index(['sex', 'chest_pain_type', 'resting_blood_pressure', 'serum_cholestoral',
       'max_heart_rate', 'exercise_induced_angina', 'oldpeak', 'slop',
       'num_major_vessels', 'thal'],
      dtype='object')

In [169]:
logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
logisticRegr.fit(X_train_model, y_train)
predictions = logisticRegr.predict(X_test_model)

In [170]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

In [171]:
print("Performance:")
print(" - accuracy = " + str(accuracy))
print(" - precision = " + str(precision))
print(" - recall = " + str(recall))
print(" - f1 = " + str(f1))

Performance:
 - accuracy = 0.7962962962962963
 - precision = 0.8387096774193549
 - recall = 0.8125
 - f1 = 0.8253968253968254


# **Wapper ma**

In [172]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [173]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)

# Set 'forward' to True for forward feature selection
sffs = SFS(
    logisticRegr,
    k_features=10,
    forward=True,  # For backward selection, set it to False
    floating=False,
    verbose=2,
    scoring='accuracy',
    cv=10,  # 10-fold cross-validation
    n_jobs=-1  # Using all available CPU cores
)

In [174]:
sffs = sffs.fit(X_train.values, y_train.values)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  13 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    0.4s finished

[2025-03-11 08:43:40] Features: 1/10 -- score: 0.767965367965368[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.4s finished

[2025-03-11 08:43:40] Features: 2/10 -- score: 0.8045454545454545[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    0.4s finished

[2025-03-11 08:43:41] Features: 3/10 -- score: 0.8415584415584416[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.4s finished

[2025-03-11 08:43:41] Features: 4/10 -- score: 0.860

In [175]:
sffs.k_feature_idx_

(1, 2, 3, 5, 6, 8, 9, 10, 11, 12)

In [176]:
# Get the selected feature names (if X is a DataFrame)
selected_feature_names = X.columns[list(sffs.k_feature_idx_)]
print("Selected Features:", selected_feature_names)

Selected Features: Index(['sex', 'chest_pain_type', 'resting_blood_pressure',
       'fasting_blood_sugar', 'resting_electro_results',
       'exercise_induced_angina', 'oldpeak', 'slop', 'num_major_vessels',
       'thal'],
      dtype='object')


In [177]:
sffs.k_score_

0.86991341991342

In [178]:
X_train_model = sffs.transform(X_train)
X_train_model

X_test_model = sffs.transform(X_test)
X_test_model

logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
logisticRegr.fit(X_train_model, y_train)
predictions = logisticRegr.predict(X_test_model)

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print("Performance:")
print(" - accuracy = " + str(accuracy))
print(" - precision = " + str(precision))
print(" - recall = " + str(recall))
print(" - f1 = " + str(f1))

Performance:
 - accuracy = 0.8148148148148148
 - precision = 0.84375
 - recall = 0.84375
 - f1 = 0.84375


**backward**

In [179]:
logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)

In [180]:
sbfs = SFS(
    logisticRegr,
    k_features=10,
    forward=True,  # For backward selection, set it to False
    floating=False,
    verbose=2,
    scoring='accuracy',
    cv=10,  # 10-fold cross-validation
    n_jobs=-1  # Using all available CPU cores
)

In [181]:
sbfs = sbfs.fit(X_train.values, y_train.values)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  13 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    0.3s finished

[2025-03-11 08:43:48] Features: 1/10 -- score: 0.767965367965368[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.4s finished

[2025-03-11 08:43:48] Features: 2/10 -- score: 0.8045454545454545[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    0.4s finished

[2025-03-11 08:43:49] Features: 3/10 -- score: 0.8415584415584416[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.4s finished

[2025-03-11 08:43:49] Features: 4/10 -- score: 0.860

In [182]:
selected_feature_names = X.columns[list(sbfs.k_feature_idx_)]
print("Selected Features:", selected_feature_names)

Selected Features: Index(['sex', 'chest_pain_type', 'resting_blood_pressure',
       'fasting_blood_sugar', 'resting_electro_results',
       'exercise_induced_angina', 'oldpeak', 'slop', 'num_major_vessels',
       'thal'],
      dtype='object')


# **Recursive Feature Elimination**

In [183]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [184]:
logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
rfe = RFE(estimator=logisticRegr, n_features_to_select=10)

In [185]:
rfe = rfe.fit(X_train, y_train)

In [186]:
X_train.columns[rfe.support_]

Index(['sex', 'chest_pain_type', 'resting_blood_pressure',
       'fasting_blood_sugar', 'resting_electro_results',
       'exercise_induced_angina', 'oldpeak', 'slop', 'num_major_vessels',
       'thal'],
      dtype='object')

In [187]:
rfe.get_support(indices=True)

array([ 1,  2,  3,  5,  6,  8,  9, 10, 11, 12])

In [188]:
X_train.columns[rfe.get_support(indices=True)]

Index(['sex', 'chest_pain_type', 'resting_blood_pressure',
       'fasting_blood_sugar', 'resting_electro_results',
       'exercise_induced_angina', 'oldpeak', 'slop', 'num_major_vessels',
       'thal'],
      dtype='object')

In [189]:
print(rfe.estimator_.coef_)

[[ 0.88546859  0.85354252  0.02374454 -1.04299823  0.37719459  0.70632996
   0.6211053   0.25044126  1.00929852  0.44087468]]


In [190]:
X_train_model = rfe.transform(X_train)
X_train_model

X_test_model = rfe.transform(X_test)
X_test_model

logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
logisticRegr.fit(X_train_model, y_train)
predictions = logisticRegr.predict(X_test_model)

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print("Performance:")
print(" - accuracy = " + str(accuracy))
print(" - precision = " + str(precision))
print(" - recall = " + str(recall))
print(" - f1 = " + str(f1))

Performance:
 - accuracy = 0.8148148148148148
 - precision = 0.84375
 - recall = 0.84375
 - f1 = 0.84375


In [191]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)


In [192]:
efs = EFS(logisticRegr,
min_features=10,
max_features=10,
scoring='accuracy',
print_progress=True,
cv=10,#10-cross validation
n_jobs=-1)#Using all availabel CPU cores

In [193]:
efs = efs.fit(X_train.values, y_train.values)
efs.subsets_


Features: 286/286

{0: {'feature_idx': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
  'cv_scores': array([0.90909091, 0.81818182, 0.90909091, 0.77272727, 0.68181818,
         0.81818182, 0.80952381, 0.85714286, 0.80952381, 0.66666667]),
  'avg_score': 0.8051948051948052,
  'feature_names': ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')},
 1: {'feature_idx': (0, 1, 2, 3, 4, 5, 6, 7, 8, 10),
  'cv_scores': array([0.86363636, 0.86363636, 0.86363636, 0.68181818, 0.72727273,
         0.77272727, 0.76190476, 0.9047619 , 0.80952381, 0.71428571]),
  'avg_score': 0.7963203463203463,
  'feature_names': ('0', '1', '2', '3', '4', '5', '6', '7', '8', '10')},
 2: {'feature_idx': (0, 1, 2, 3, 4, 5, 6, 7, 8, 11),
  'cv_scores': array([0.81818182, 0.72727273, 0.95454545, 0.81818182, 0.77272727,
         0.90909091, 0.80952381, 0.80952381, 0.76190476, 0.76190476]),
  'avg_score': 0.8142857142857143,
  'feature_names': ('0', '1', '2', '3', '4', '5', '6', '7', '8', '11')},
 3: {'feature_idx': (0, 1, 2, 3, 4, 5, 6, 7, 8, 12),
  'cv_s

In [194]:
efs.best_score_

0.8742424242424243

In [195]:
efs.best_idx_

(1, 2, 3, 5, 7, 8, 9, 10, 11, 12)

In [196]:
# Get the selected feature names (if X is a DataFrame)
selected_feature_names = X.columns[list(efs.best_idx_)]
print("Selected Features:", selected_feature_names)


Selected Features: Index(['sex', 'chest_pain_type', 'resting_blood_pressure',
       'fasting_blood_sugar', 'max_heart_rate', 'exercise_induced_angina',
       'oldpeak', 'slop', 'num_major_vessels', 'thal'],
      dtype='object')


In [197]:
X_train_model = efs.transform(X_train)
X_train_model

X_test_model = efs.transform(X_test)
X_test_model

logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
logisticRegr.fit(X_train_model, y_train)
predictions = logisticRegr.predict(X_test_model)

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print("Performance:")
print(" - accuracy = " + str(accuracy))
print(" - precision = " + str(precision))
print(" - recall = " + str(recall))
print(" - f1 = " + str(f1))


Performance:
 - accuracy = 0.8148148148148148
 - precision = 0.84375
 - recall = 0.84375
 - f1 = 0.84375


# **Embedded Methods**

In [198]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [199]:
# builds 100 trees
rf = RandomForestClassifier(n_estimators=100, random_state=2025)
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=2025)

In [200]:
importances = rf.feature_importances_
print(importances)

[0.07516128 0.03446577 0.13410932 0.07523985 0.08597789 0.00768271
 0.02034116 0.12056793 0.05201174 0.13136909 0.03705368 0.09897439
 0.1270452 ]


In [201]:
indices = np.argsort(importances)[::-1]
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, X_train.columns[indices[f]], importances[indices[f]]))

 1) chest_pain_type                0.134109
 2) oldpeak                        0.131369
 3) thal                           0.127045
 4) max_heart_rate                 0.120568
 5) num_major_vessels              0.098974
 6) serum_cholestoral              0.085978
 7) resting_blood_pressure         0.075240
 8) age                            0.075161
 9) exercise_induced_angina        0.052012
10) slop                           0.037054
11) sex                            0.034466
12) resting_electro_results        0.020341
13) fasting_blood_sugar            0.007683


In [202]:
sfm = SelectFromModel(rf, threshold=0.04)
sfm.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(random_state=2025),
                threshold=0.04)

In [203]:
rf_feature_boolean_selector = sfm.get_support()
rf_feature_boolean_selector

array([ True, False,  True,  True,  True, False, False,  True,  True,
        True, False,  True,  True])

In [204]:
rf_feature_set = X_train.columns[rf_feature_boolean_selector]
rf_feature_set

Index(['age', 'chest_pain_type', 'resting_blood_pressure', 'serum_cholestoral',
       'max_heart_rate', 'exercise_induced_angina', 'oldpeak',
       'num_major_vessels', 'thal'],
      dtype='object')

In [205]:
X_train_model = sfm.transform(X_train)
X_train_model

X_test_model = sfm.transform(X_test)
X_test_model

array([[6.90e+01, 1.00e+00, 1.40e+02, 2.39e+02, 1.51e+02, 0.00e+00,
        1.80e+00, 2.00e+00, 3.00e+00],
       [5.70e+01, 4.00e+00, 1.28e+02, 3.03e+02, 1.59e+02, 0.00e+00,
        0.00e+00, 1.00e+00, 3.00e+00],
       [5.80e+01, 2.00e+00, 1.36e+02, 3.19e+02, 1.52e+02, 0.00e+00,
        0.00e+00, 2.00e+00, 3.00e+00],
       [6.90e+01, 3.00e+00, 1.40e+02, 2.54e+02, 1.46e+02, 0.00e+00,
        2.00e+00, 3.00e+00, 7.00e+00],
       [4.80e+01, 3.00e+00, 1.24e+02, 2.55e+02, 1.75e+02, 0.00e+00,
        0.00e+00, 2.00e+00, 3.00e+00],
       [5.60e+01, 4.00e+00, 1.32e+02, 1.84e+02, 1.05e+02, 1.00e+00,
        2.10e+00, 1.00e+00, 6.00e+00],
       [4.30e+01, 3.00e+00, 1.30e+02, 3.15e+02, 1.62e+02, 0.00e+00,
        1.90e+00, 1.00e+00, 3.00e+00],
       [6.50e+01, 1.00e+00, 1.38e+02, 2.82e+02, 1.74e+02, 0.00e+00,
        1.40e+00, 1.00e+00, 3.00e+00],
       [5.20e+01, 4.00e+00, 1.12e+02, 2.30e+02, 1.60e+02, 0.00e+00,
        0.00e+00, 1.00e+00, 3.00e+00],
       [6.50e+01, 4.00e+00, 1.50e+02,

In [206]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
logisticRegr.fit(X_train_model, y_train)

LogisticRegression(max_iter=1000)

In [207]:
predictions = logisticRegr.predict(X_test_model)

In [208]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print("Performance:")
print(" >accuracy = " + str(accuracy))
print(" >precision = " + str(precision))
print(" >recall = " + str(recall))
print(" >f1 = " + str(f1))


Performance:
 >accuracy = 0.7777777777777778
 >precision = 0.8333333333333334
 >recall = 0.78125
 >f1 = 0.8064516129032258


# **Feature Extraction**

In [209]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
df = pd.read_csv(url, header=None, sep=',')

In [210]:
df.shape

(178, 14)

In [212]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [213]:
 X = df.iloc[:, 1:].values
 y = df.iloc[:,0].values

In [214]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{1: 59, 2: 71, 3: 48}

In [215]:
df[0].value_counts()

,count
0,
2,71
1,59
3,48




```
# Classifation
```



In [216]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y, random_state=0)

In [217]:
from sklearn.preprocessing import StandardScaler
# standardize the features
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [218]:
from sklearn.decomposition import PCA
pca = PCA(n_components=None)

In [219]:
X_train_pca = pca.fit_transform(X_train_std)

In [220]:
 #Eigenvalues
pca.explained_variance_

array([4.84274532, 2.41602459, 1.54845825, 0.96120438, 0.84166161,
       0.6620634 , 0.51828472, 0.34650377, 0.3131368 , 0.21357215,
       0.1808613 , 0.15362835, 0.10754642])

In [221]:
pca.explained_variance_ratio_

array([0.36951469, 0.18434927, 0.11815159, 0.07334252, 0.06422108,
       0.05051724, 0.03954654, 0.02643918, 0.02389319, 0.01629614,
       0.01380021, 0.01172226, 0.00820609])

In [222]:
pca.components_

array([[ 1.37242175e-01, -2.47243265e-01,  2.54515927e-02,
        -2.06945084e-01,  1.54365821e-01,  3.93769523e-01,
         4.17351064e-01, -3.05728961e-01,  3.06683469e-01,
        -7.55406578e-02,  3.26132628e-01,  3.68610222e-01,
         2.96696514e-01],
       [ 5.03034778e-01,  1.64871190e-01,  2.44564761e-01,
        -1.13529045e-01,  2.89745182e-01,  5.08010391e-02,
        -2.28733792e-02,  9.04888470e-02,  8.35232677e-03,
         5.49775805e-01, -2.07164328e-01, -2.49025357e-01,
         3.80229423e-01],
       [-1.37748734e-01,  9.61503863e-02,  6.77775667e-01,
         6.25040550e-01,  1.96135481e-01,  1.40310572e-01,
         1.17053859e-01,  1.31217777e-01,  3.04309008e-02,
        -7.99299713e-02,  5.30591506e-02,  1.32391030e-01,
        -7.06502178e-02],
       [-3.29610003e-03,  5.62646692e-01, -1.08977111e-01,
         3.38187002e-02, -3.67511070e-01,  2.40245127e-01,
         1.87053299e-01, -2.29262234e-02,  4.96262330e-01,
         1.06482939e-01, -3.69053747e

In [223]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

In [224]:
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

In [225]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='auto', solver='liblinear')

In [226]:
lr.fit(X_train_pca, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(multi_class='auto', solver='liblinear')

In [227]:
predictions = lr.predict(X_test_pca)

In [228]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions, labels=[1, 2, 3])

array([[15,  3,  0],
       [ 1, 20,  0],
       [ 0,  0, 15]])